In [13]:
PROJECT_ID = 'gcp-ccai-auto-ml-contactcenter'
REGION= "europe-west3"
REPO_NAME = "repo-demo3"
SERVICE_ACCOUNT = "944308723981-compute@developer.gserviceaccount.com"
BUCKET = "ccai-storage"
PIPELINE_NAME = "automl_pipeline"
YAML_NAME = f"{PIPELINE_NAME}.yml"
PIPELINE_ROOT = f"gs://{BUCKET}/pipeline_root/"
DISPLAY_NAME = PIPELINE_NAME.replace("_", "-")
NOTEBOOK = "automl"
DATANAME = "datasetnlp"
FOLDER = 'make_prediction'
PROJECT_ID = 'gcp-ccai-auto-ml-contactcenter'
TABLE_ID = "testdatabq"
TEXT_COLUMN = 'body_pre'
LOCATION = "europe-west3"
RANDOM_SEED=123
#BQ_SOURCE = "bq://gcp-ccai-auto-ml-contactcenter.datasetnlp.stepfinalbq"

OUTPUT_FINAL = 'step_final_bq.parquet'

# Resources
DEPLOY_COMPUTE = 'n1-standard-4'

In [14]:
APPLICATION_DIR = "src"

In [15]:

!pwd

/home/jupyter/gcp-ccai-auto-ml-contactcenter/End-to-end-AutoML-Topic-Classification/Get_Predictions_API/serving_image/src


In [16]:
!ls

Dockerfile   app.py		     processing
__pycache__  gcs_parquet_loader.log  requirements.txt


In [17]:
%%writefile {APPLICATION_DIR}/Dockerfile

# Use the official Python image
FROM us-docker.pkg.dev/deeplearning-platform-release/gcr.io/base-cu113.py310
 
# Set the working directory in the container
WORKDIR /
 
ENV HOST 0.0.0.0

# Copy the requirements file into the container
COPY requirements.txt .
 
# Install dependencies
RUN pip install -r requirements.txt
 
# Copy the rest of the application code into the container
COPY . /
 
# Command to run the FastAPI application
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8081"]

EXPOSE 8081

Writing src/Dockerfile


FileNotFoundError: [Errno 2] No such file or directory: 'src/Dockerfile'

In [18]:
! gcloud auth configure-docker


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "europe-west3-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for all GCR repositories.
gcloud credential helpers already registered correctly.


In [19]:
!gcloud artifacts repositories create $REPO_NAME --repository-format=docker \
--location=$REGION --description="Docker repository"

ERROR: (gcloud.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists


In [20]:
IMAGE_URI = (
    f"{LOCATION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}/fast_api_automl_serving:latest"
)

In [21]:
IMAGE_URI

'europe-west3-docker.pkg.dev/gcp-ccai-auto-ml-contactcenter/repo-demo3/fast_api_automl_serving:latest'

In [22]:
cd $APPLICATION_DIR

[Errno 2] No such file or directory: 'src'
/home/jupyter/gcp-ccai-auto-ml-contactcenter/End-to-end-AutoML-Topic-Classification/Get_Predictions_API/serving_image/src


In [23]:
! docker build ./ -t $IMAGE_URI

Sending build context to Docker daemon  113.2kB
Step 1/8 : FROM us-docker.pkg.dev/deeplearning-platform-release/gcr.io/base-cu113.py310
 ---> 039eb2919b37
Step 2/8 : WORKDIR /
 ---> Using cache
 ---> eb5e1d2a1fa8
Step 3/8 : ENV HOST 0.0.0.0
 ---> Using cache
 ---> 355be3dc3b47
Step 4/8 : COPY requirements.txt .
 ---> 818648b146a1
Step 5/8 : RUN pip install -r requirements.txt
 ---> Running in 67aaae4be761
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 155.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 10.8 MB/s e

In [24]:
! docker push $IMAGE_URI

The push refers to repository [europe-west3-docker.pkg.dev/gcp-ccai-auto-ml-contactcenter/repo-demo3/fast_api_automl_serving]

6e548201: Preparing 
9822fda4: Preparing 
5b289742: Preparing 
0840b95b: Preparing 
625a528c: Preparing 
1351140a: Preparing 
bf18a086: Preparing 
82771da2: Preparing 
57952521: Preparing 
6a5b98f7: Preparing 
bd61aa20: Preparing 
5ef9c8b3: Preparing 
a870a840: Preparing 
43031e88: Preparing 
ab4e5392: Preparing 
07dc0b52: Preparing 
f18a086: Waiting g 
379a4282: Preparing 
a5b98f7: Waiting g 
6aad04a1: Preparing 
dfc3b824: Preparing 
4c2a35b9: Preparing 
2771da2: Waiting g 
7952521: Waiting g 
cd77acfc: Preparing 
b784db22: Preparing 
f18a086: Preparing 
88822c42: Preparing 
95b7aaed: Preparing 
ea69765: Waiting g 
2c163a83: Preparing 
13d8681b: Preparing 
d61aa20: Waiting g 
ef9c8b3: Waiting g 
aad04a1: Waiting g 
870a840: Waiting g 
fc3b824: Waiting g 
30c0e62e: Preparing 
c2a35b9: Waiting g 
3031e88: Waiting g 
822fda4: Pushed   2.158GB/2.139GBlatest: diges